### Adam算法

Adam算法可以看做是RMSProp算法和动量法的结合。

Adam算法使用了动量变量$v_t$和RMSProp算法中的状态变量$s_t$.

对于动量变量$v_t$,时间步为0时将它初始化为0，给定超参数$0 \le \beta_1 \le 1$(算法作者建议设为0.9),时间步为t时，有

$v_t \leftarrow \beta_1 v_{t-1}+(1-\beta_1)g_t$

对于状态变量$s_t$,时间步为0时将它初始化为0，给定超参数$0 \le \beta_2 \le 1$(算法作者建议0.999),时间步为t时，有

$s_t \leftarrow \beta_2 s_{t-1}+(1-\beta_2)g_t \bigodot g_t$

当t较小时，过去各时间步小批量随机梯度权值之和会较小，为了消除这样的影响，在Adam中使用了偏差修正，即

$v_t \leftarrow \frac{v_t}{1-\beta_1^{t}}$

$s_t \leftarrow \frac{s_t}{1-\beta_2^{t}}$

接下来计算$g_t^{'}$

$g_t^{'} \leftarrow \frac{\eta v_t}{\sqrt{s_t+eps}}$

最后迭代更新时有:

$x_t \leftarrow x_{t-1}-g_t^{'}$

### 从零实现

In [1]:
import utils
import torch

def init_adam_states():
    v_w,v_b=torch.zeros((features.shape[1],1),dtype=torch.float32),torch.zeros(1,dtype=torch.float32)
    s_w,s_b=torch.zeros((features.shape[1],1),dtype=torch.float32),torch.zeros(1,dtype=torch.float32)
    return ((v_w,v_b),(s_w,s_b))

def adam(params,states,hyperparams):
    beta1,beta2,eps=0.9,0.999,1e-6
    for p,(v,s) in zip(params,states):
        v[:]=beta1*v+(1-beta1)*p.grad.data
        s[:]=beta2*s+(1-beta2)*p.grad.data**2
        v_bias_corr=v/(1-beta1**hyperparams['t'])
        s_bias_corr=s/(1-beta2**hyperparams['t'])
        p.data-=hyperparams['lr']*v_bias_corr/(torch.sqrt(s_bias_corr)+eps)
    hyperparams['t']+=1
    

In [ ]:
utils.train_ch7(adam,init_adam_states(),{'lr':0.01,'t':1},features,labels)

### 简洁实现

In [ ]:
utils.train_pytorch_ch7(torch.optim.Adam,{'lr':0.01},features,labels)